In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from random import randint
NUM_SAMPLES = 2

model_name_or_path = "TheBloke/CapybaraHermes-2.5-Mistral-7B-GPTQ"
revision = "main"

model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision=revision)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
prompt = "give me one sample"
#prompt_template = f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
from datasets import load_dataset

flute_ds = load_dataset("ColumbiaNLP/FLUTE")

In [3]:
def get_samples_string(ds):
    samples = ""
    for i in range(NUM_SAMPLES):
        sample = ds[randint(0, len(ds))]
        samples += f"""\nsample {i+1}\n<Hypothesis>: {sample['hypothesis']}\n<Premise>: {sample['premise']}\n<Label>: {sample['label']}\n<Explanation>: {sample['explanation']}\n"""
    return samples

In [50]:
def get_model_output(ds, specific_msg):
    samples = get_samples_string(ds)
    system_message = specific_msg + samples + "\n"
    prompt_template = f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"

    input_ids = tokenizer(5*[prompt_template], return_tensors='pt').input_ids.cuda()
    output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
    return output

# Sarcasm
There's only contradiction here

In [51]:
sarcasm_ds = flute_ds['train'].filter(lambda x: x['type'] == "Sarcasm")

## Contradiction

In [58]:
specific_msg = """You should provide examples of sarcastic sentences (hypothesis) that refer to a situation (premise) and explain why there is a contradiction (explanation) 
Use these samples of hypothesis, premise, label, and explanation and adhere to this format when replying
"""

output = get_model_output(sarcasm_ds, specific_msg)

In [62]:
print(tokenizer.decode(output[0]))

<s><|im_start|> system
You should provide examples of sarcastic sentences (hypothesis) that refer to a situation (premise) and explain why there is a contradiction (explanation) 
Use these samples of hypothesis, premise, label, and explanation and adhere to this format when replying

sample 1
<Hypothesis>: It's awesome that my boss can afford a new Mercedes when I can only afford public transport
<Premise>: My boss just bought a new Mercedes, and I can only afford public transport
<Label>: Contradiction
<Explanation>: A new Mercedes car is a very expensive purchase and so the fact that someone's boss can afford it implies that they are very wealthy, while someone who can only afford public transport is likely living paycheck to paycheck and feels jealous.

sample 2
<Hypothesis>: Cleaning my toilet always feels like the most fun thing I could ever do and I feel excited every time
<Premise>: I clean my house every week and last time I had to clean my toilet, I almost threw up.
<Label>: C

# Idiom

In [64]:
idiom_ds = flute_ds['train'].filter(lambda x: x['type'] == "Idiom")
con_idiom_ds = idiom_ds.filter(lambda x: x['label'] == "Contradiction")
ent_idiom_ds = idiom_ds.filter(lambda x: x['label'] == "Entailment")

Filter:   0%|          | 0/1768 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1768 [00:00<?, ? examples/s]

## Contradiction

In [89]:
specific_msg = """You should provide examples of idiomatic expressions being incorrectly used (hypothesis) that refer to a situation (premise) and explain why they are not the correct idiom to be used in that setting (explanation) 
Use these samples of hypothesis, premise, label, and explanation and adhere to this format when replying
"""

output = get_model_output(con_idiom_ds, specific_msg)

In [94]:
print(tokenizer.decode(output[4]))

<s><|im_start|> system
You should provide examples of idiomatic expressions being incorrectly used (hypothesis) that refer to a situation (premise) and explain why they are not the correct idiom to be used in that setting (explanation) 
Use these samples of hypothesis, premise, label, and explanation and adhere to this format when replying

sample 1
<Hypothesis>: They had suddenly gone from comfortable to living on a shoestring and being behind on something as important as their mortgage.
<Premise>: They had suddenly gone from comfortable to living very lavishly and within a large financial budget and being behind on something as important as their mortgage.
<Label>: Contradiction
<Explanation>: To be on a shoestring means to be on a very tight or limited financial budget, but in this context they were living very lavishly

sample 2
<Hypothesis>: By midnight, mia was feeling the effects of the alcohol, and she slowed her intake while the others continued to whoop it up.
<Premise>: By m

## Entailment

In [95]:
specific_msg = """You should provide examples of idiomatic expressions being correctly used (hypothesis) that refer to a situation (premise) and explain why they are the correct idiom to be used in that setting (explanation) 
Use these samples of hypothesis, premise, label, and explanation and adhere to this format when replying
"""

output = get_model_output(ent_idiom_ds, specific_msg)

In [96]:
print(tokenizer.decode(output[4]))

<s><|im_start|> system
You should provide examples of idiomatic expressions being correctly used (hypothesis) that refer to a situation (premise) and explain why they are the correct idiom to be used in that setting (explanation) 
Use these samples of hypothesis, premise, label, and explanation and adhere to this format when replying

sample 1
<Hypothesis>: Hopefully, the claim would slip through the net, but if it didn't she was in even more trouble.
<Premise>: Hopefully, the claim would go unnoticed or be neglected, but if it didn't she was in even more trouble.
<Label>: Entailment
<Explanation>: To slip through the net means to go unnoticed or be neglected, and in this context it is hoped that the claim will go unnoticed.

sample 2
<Hypothesis>: "Good enough" Grond said now lead the way.
<Premise>: "Good enough" Grond said now go ahead of me and show me the way.
<Label>: Entailment
<Explanation>: To lead the way means to guide or direct someone somewhere by proceeding ahead of them,

# Metaphor

# Simile

# Creative Paraphrase